<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/team5_lab09_model/heart_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<b><br><font face = "Gill Sans" size = "4"><center>Heart attack prediction with logistic regression</center></font></b>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,091 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,431 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,208 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease

<span style="font-family:Gill Sans">
<h1>Hearth attach prediction with logisic regression</h1></span>

Let us use PySpark to predict if a person can suffer a heart attack based on some medical information.

Download the dataset from [Kaggle](https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, FloatType, IntegerType

# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ITESO-LogisticRegression").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

heart_prediction_schema = StructType([
    StructField("male", IntegerType(), True),
    StructField("age", IntegerType(), True),
    StructField("education", IntegerType(), True),
    StructField("currentSmoker", IntegerType(), True),
    StructField("cigsPerDay", IntegerType(), True),
    StructField("BPMeds", IntegerType(), True),
    StructField("prevalentStroke", IntegerType(), True),
    StructField("prevalentHyp", IntegerType(), True),
    StructField("diabetes", IntegerType(), True),
    StructField("totChol", FloatType(), True),
    StructField("sysBP", FloatType(), True),
    StructField("diaBP", FloatType(), True),
    StructField("BMI", FloatType(), True),
    StructField("heartRate", IntegerType(), True),
    StructField("glucose", IntegerType(), True),
    StructField("TenYearCHD", IntegerType(), True)
])

heart_df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/framingham.csv").\
    schema(heart_prediction_schema).\
    load()
heart_df.printSchema()
heart_df.show(n=10, truncate=False)

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: float (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+----

In [ ]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|      105|            0|        29|    53|              0|           0|       0|     50|    0|    0| 19|        1|    388|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [ ]:
# Remove rows with at least one value as null
heart_df = heart_df.dropna()

In [ ]:
heart_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in heart_df.columns]).show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+
|   0|  0|        0|            0|         0|     0|              0|           0|       0|      0|    0|    0|  0|        0|      0|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+---+---------+-------+----------+



In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

# Assemble the features into a single vector column
assembler = VectorAssembler(inputCols=["male","age","education","currentSmoker","cigsPerDay","BPMeds","prevalentStroke","prevalentHyp","diabetes","totChol","sysBP","diaBP","BMI","heartRate","glucose"], outputCol="features")
data_with_features = assembler.transform(heart_df).select("TenYearCHD", "features")

#data_with_features.show(n=5)
# Split the data into training and test sets 70% training data and 30% testing data
train, test = data_with_features.randomSplit([0.7, 0.3], seed=57)

train.show(n=5)
test.show(n=5)

+----------+--------------------+
|TenYearCHD|            features|
+----------+--------------------+
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
+----------+--------------------+
only showing top 5 rows

+----------+--------------------+
|TenYearCHD|            features|
+----------+--------------------+
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
|         0|(15,[1,2,9,10,11,...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.classification import LogisticRegression
# Create a logistic regression model
lr = LogisticRegression(maxIter=10, regParam=0.01)
lr = LogisticRegression(featuresCol="features", labelCol="TenYearCHD")

# Train to get the model
lr_model = lr.fit(train)

In [ ]:
# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

# Make predictions on the test set

# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test)

# Show predictions
predictions.select("features", "prediction", "probability").show()

# Stop Spark session
spark.stop()

Coefficients: [0.6378773480532327,0.062395508152567634,-0.012446089746229677,0.0562932799243091,0.014740607924864091,0.32964839757063386,0.576104308672386,0.2126224654121739,0.19174996969202535,0.0009492877303977642,0.017508810698592142,-0.0036997324537733313,0.001362093080941249,-0.00226359081418126,0.008170615824907128]
+--------------------+----------+--------------------+
|            features|prediction|         probability|
+--------------------+----------+--------------------+
|(15,[1,2,9,10,11,...|       0.0|[0.98254785867922...|
|(15,[1,2,9,10,11,...|       0.0|[0.97722045111776...|
|(15,[1,2,9,10,11,...|       0.0|[0.97796036917776...|
|(15,[1,2,9,10,11,...|       0.0|[0.98092929414614...|
|(15,[1,2,9,10,11,...|       0.0|[0.97350378227493...|
|(15,[1,2,9,10,11,...|       0.0|[0.97110978611260...|
|(15,[1,2,9,10,11,...|       0.0|[0.98173207239612...|
|(15,[1,2,9,10,11,...|       0.0|[0.98092996529309...|
|(15,[1,2,9,10,11,...|       0.0|[0.95229725542918...|
|(15,[1,2,9,10,1